In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score # confusion_matrix, 

from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import OneHotEncoder, StandardScaler  , PolynomialFeatures
from sklearn.compose import ColumnTransformer

# from sklearn.linear_model import LogisticRegression 
from xgboost import XGBClassifier 



from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer


k_target        = "converted"
k_samples_ratio = 100/100   # percentage of observations to be taken into account. Pass 100/100 for final testing 
k_test_size     = 20/100    # see train_test_split
k_random_state  = 42        # you know why...
header          = "conversion_data_test_predictions_"
author          = "PHILIPPE"


In [2]:
df = pd.read_csv('./assets/conversion_data_train.csv')
df.head()

,country,age,new_user,source,total_pages_visited,converted
0,China,22,1,Direct,2,0
1,UK,21,1,Ads,3,0
2,Germany,20,0,Seo,14,1
3,US,23,1,Seo,3,0
4,US,28,1,Direct,3,0


In [3]:
# print(f"shape : {df.shape}\n")
# print()

# display(df.describe(include="all").T)

In [4]:
# print(df.info(), "\n")

In [5]:
# print(f"Number of null val :")
# print(100 * df.isnull().sum() / df.shape[0])
# # print (df.isnull().any().any())

In [6]:
# print("Duplicates     : ", df.duplicated().sum())
# print("Col duplicated : ", df.columns.duplicated() )

# print()
# print("Unique countries : ", df["country"].unique())
# print("Unique sources   : ", df["source"].unique())

# # print(df["col_name"].value_counts())
# # print(df.isnull().sum().sort_values(ascending=False).head(11))
# df[k_target].value_counts()


## Preprocessing on df

In [7]:
# Uniquement sur les données de train
# PAS sur les données de test (validation) ou les données de test jamais vues
# ATTENTION : 
# Il est sans doute préférable de minorer l'influence des observations redontantes

# indices_redondants = [0, 1, 5, 10]  # Exemple d'indices de données redondantes

# # Calculer les poids des échantillons en inversant leur fréquence
# # Pour pondérer à la baisse les échantillons redondants, vous pouvez diviser 1 par le nombre d'occurrences de chaque échantillon
# weights = np.ones(len(y_train))
# for idx in indices_redondants:
#     weights[idx] = 1.0 / indices_redondants.count(idx)

# Créer un objet DMatrix pour les données d'entraînement avec les poids des échantillons
# dtrain = xgb.DMatrix(data=X_train, label=y_train, weight=weights)

# https://xgboost.readthedocs.io/en/stable/parameter.html
# Réutiliser aussi les paramètres issues du GridSearchCV
# params = {
#     'objective': 'binary:logistic',  # pour un problème de classification binaire
#     'eval_metric': 'auc'  # Métrique d'évaluation
# }

# # Entraîner le modèle XGBoost
# model = xgb.train(params=params, dtrain=dtrain, num_boost_round=100)






# def add_weight_col(df):
  
#   # print(f"shape : {df.shape}\n")
#   # df.drop_duplicates(inplace=True)
#   # print(f"shape : {df.shape}\n")

#   # print(f"shape : {df.shape}")

#   # Créer une colonne avec l poids des doublons
#   # Supprimer les doublons
#   df_no_duplicates = df.drop_duplicates()

#   # Compter le nombre d'occurrences de chaque ligne dans le DataFrame d'origine
#   counts = df.groupby(df.columns.tolist()).size().reset_index(name='poids')

#   # Fusionner la colonne occurences avec le df sans doublons
#   df = pd.merge(df_no_duplicates, counts, on=df.columns.tolist(), how='left')

#   # print(f"shape : {df.shape}")
#   # df.to_csv("./assets/pourbench_colab.csv")
#   return df  






In [8]:
# Removing outliers
# Uniquement sur les données de train
# PAS sur les données de test (validation) ou les données de test jamais vues

# print(df.shape)

# numeric_columns = df[["age", "total_pages_visited"]]
# print(type(numeric_columns))
# # numeric_columns = ["age	new_user", "total_pages_visited"]

# # 1. Calculez l'IQR pour chaque colonne numérique
# Q1 = numeric_columns.quantile(0.25)
# Q3 = numeric_columns.quantile(0.75)
# IQR = Q3 - Q1

# # 2. Calculez les limites supérieure et inférieure
# lower_bound = Q1 - 1.5 * IQR
# upper_bound = Q3 + 1.5 * IQR

# # Créez un masque pour identifier les lignes avec des valeurs aberrantes dans les colonnes numériques
# outliers_mask = ((numeric_columns < lower_bound) | (numeric_columns > upper_bound)).any(axis=1)

# # Filtrez les lignes avec des valeurs aberrantes uniquement dans les colonnes numériques
# df = df[~outliers_mask]
# print(df.shape)

# df.head()

In [9]:
# print(f"shape : {df.shape}")

# # df = add_weight_col(df)  

# print(f"shape : {df.shape}")
 

# # On peut ici limiter la taille de df pour aller plus vite par exemple  
# # df = df.sample(int(k_samples_ratio*len(df)))
# # df = df.iloc[:int(k_samples_ratio*len(df))]


In [10]:

# Split entre X et y
X = df.drop(columns = k_target)
y = df[k_target]

print("X :")
print(X.head())
print(X.shape)
print()

print("y :")
print(y.head())

# Des dataframes
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=k_test_size, random_state=k_random_state, stratify = y)

print(X_train.shape)
print(type(X_train))


X :
   country  age  new_user  source  total_pages_visited
0    China   22         1  Direct                    2
1       UK   21         1     Ads                    3
2  Germany   20         0     Seo                   14
3       US   23         1     Seo                    3
4       US   28         1  Direct                    3
(284580, 5)

y :
0    0
1    0
2    1
3    0
4    0
Name: converted, dtype: int64
(227664, 5)
<class 'pandas.core.frame.DataFrame'>


In [11]:
numeric_features = X.select_dtypes(include="number").columns
print(numeric_features)

categorical_features = X.select_dtypes(exclude="number").columns
print(categorical_features)

numeric_transformer = Pipeline(
  steps=[
    # ("imputer_num", SimpleImputer(strategy="mean")),
    ("scaler_num", StandardScaler()),
  ]
)

categorical_transformer = Pipeline(
  steps=[
      # ("imputer_cat", SimpleImputer(strategy="most_frequent")),  
      # ("imputer_cat", SimpleImputer(fill_value="missing", strategy="constant")),  
      ("encoder_cat", OneHotEncoder(drop="first")),                 
      # ("encoder_cat", OneHotEncoder(handle_unknown='ignore', sparse=False)),                 
    ]
  )

preprocessor = ColumnTransformer(
  transformers=[
      ("num", numeric_transformer,     numeric_features),
      ("cat", categorical_transformer, categorical_features),
    ]
  )



Index(['age', 'new_user', 'total_pages_visited'], dtype='object')
Index(['country', 'source'], dtype='object')


In [12]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

# des nd array
print(X_train[0:5].round(3))
print(X_train.shape)
print(type(X_train))


[[-1.277  0.676 -0.262  0.     0.     1.     0.     0.   ]
 [-0.189  0.676 -0.561  0.     0.     0.     0.     0.   ]
 [ 0.657 -1.479 -0.561  0.     0.     1.     0.     1.   ]
 [-0.914  0.676  0.934  0.     0.     1.     0.     1.   ]
 [ 1.262  0.676 -0.561  0.     1.     0.     0.     0.   ]]
(227664, 8)
<class 'numpy.ndarray'>


In [13]:
# param_grid = {
#     'learning_rate': [0.1, 0.01],
#     'n_estimators': [100, 200],
#     'max_depth': [3, 5],
#     'subsample': [0.8, 0.9],
#     'colsample_bytree': [0.8, 0.9],
#     'scale_pos_weight': [30, 20, 10, 1] 
                                        
# }

# Tres très loong. J'ai arrêté aprs 441 min. A refaire une nuit
# https://machinelearningmastery.com/xgboost-for-imbalanced-classification/
# param_grid = {
#     'learning_rate': [0.01, 0.05, 0.1],
#     'n_estimators': [200, 400, 600],
#     'max_depth': [6, 8, 10],
#     'subsample': [0.8, 0.9, 1.0],
#     'colsample_bytree': [0.8, 0.9, 1.0],
#     'scale_pos_weight': [10, 20, 30]  # Control the balance of positive and negative weights, useful for unbalanced classes. 
# }                                     # A typical value to consider: sum(negative instances) / sum(positive instances). 


# Composite estimators
# https://scikit-learn.org/stable/modules/compose.html
pipe = Pipeline(steps=[
    # ("poly", PolynomialFeatures()),
    ("xgb", XGBClassifier())
])

# param_grid = {
#     "poly__degree"            : [1],
#     'xgb__learning_rate'      : [0.1, 1],
#     'xgb__n_estimators'       : [400, 600],
#     #'xgb__max_depth'         : [6, 8],
#     #'xgb__subsample'         : [0.8, 0.9],
#     #'xgb__colsample_bytree'  : [0.8, 0.9],
#    'xgb__scale_pos_weight'    : [10, 20]  # Control the balance of positive and negative weights, useful for unbalanced classes. 
# }                                         # A typical value to consider: sum(negative instances) / sum(positive instances). 


# https://towardsdatascience.com/a-guide-to-xgboost-hyperparameters-87980c7f44a9
param_grid = {
    # "poly__degree"          : [1],
    "xgb__objective"          : ["binary:logistic"],
    'xgb__n_estimators'       : [600],        # default 100
    'xgb__subsample'          : [1],            # default 1 [0,1]
    'xgb__max_depth'          : [6],          # default 6
    'xgb__learning_rate'      : [0.4],         # default 0.3
    'xgb__gamma'               : [0],         # default 0 [0-1[ regulation
    'xgb__reg_alpha'           : [0],         # default 0 L1 regulation
    'xgb__lambda'               : [1],         # default 1 L2 regulation
    # 'xgb__scale_pos_weight'   : [0.5 + i * 0.5 for i in range(20)],         # ici de 0.5 à 10. default 1 Neg / Pos instances Control the balance of positive and negative weights, useful for unbalanced classes



    # 'xgb__colsample_bytree'   : [1],         # default 1 specify the fraction of columns to be subsampled.
    # 'xgb__colsample_bylevel'   : [1],         # default 1 specify the fraction of columns for training at each level


}


grid_search = GridSearchCV(estimator=pipe, param_grid=param_grid, scoring="f1", verbose = 3, n_jobs = -1, cv = 5) #refit=True,     
grid_search.fit(X_train, y_train)

print(grid_search.best_params_)

best_estimator = grid_search.best_estimator_
y_pred = best_estimator.predict(X_test)

print(f"f1 \t\t precision \t recall")
print(f"{f1_score(y_test,  y_pred):.6f} \t {precision_score(y_test,  y_pred):.6f} \t {recall_score(y_test,  y_pred):.6f}")



Fitting 5 folds for each of 1 candidates, totalling 5 fits
{'xgb__gamma': 0, 'xgb__lambda': 1, 'xgb__learning_rate': 0.4, 'xgb__max_depth': 6, 'xgb__n_estimators': 600, 'xgb__objective': 'binary:logistic', 'xgb__reg_alpha': 0, 'xgb__subsample': 1}
f1 		 precision 	 recall
0.751957 	 0.840511 	 0.680283


## Entrainement sur l'ensemble du jeu de données 
* sans le diviser en train et test
* L'idée c'est d'utiliser un max d'observations pour ajuster les paramètres du modèle

In [14]:
#  Y a plus de train_test_split
X = df.drop(columns = k_target)
y = df[k_target]

# print(X.shape)
# print(type(X))


In [15]:
X = preprocessor.fit_transform(X)

# print(X[0:5].round(3))
# print(X.shape)
# print(type(X))


In [16]:
# clf = XGBClassifier()

# clf.set_params(**best_params)
# clf.fit(X_train, y_train)


In [17]:
y_pred = best_estimator.predict(X)

print(f"f1 \t\t precision \t recall")
print(f"{f1_score(y,  y_pred):.6f} \t {precision_score(y,  y_pred):.6f} \t {recall_score(y,  y_pred):.6f}")

f1 		 precision 	 recall
0.783811 	 0.874883 	 0.709913


## Predictions sur le jeu sans label

In [18]:
df_no_labels = pd.read_csv('./assets/conversion_data_test.csv')
print(type(df_no_labels))
print(df_no_labels.shape)

# X_no_labels = add_weight_col(df_no_labels) 


X_no_labels = df_no_labels
X_no_labels = preprocessor.transform(X_no_labels)

# clf = XGBClassifier()
# clf.set_params(**best_params)
y_no_labels = best_estimator.predict(X_no_labels) 

# print(type(X_no_labels))
# print(X_no_labels.shape)

# print(type(X_no_labels))
# print(X_no_labels.shape)
# print(X_no_labels[0:5,:].round(3))



<class 'pandas.core.frame.DataFrame'>
(31620, 5)


In [19]:
# sauvegarde des prédictions

data = {
  'converted': y_no_labels
}

Y_predictions = pd.DataFrame(columns=['converted'], data=data)

trailer         = datetime.now().strftime("%Y%m%d_%H%M%S")
out_file = "./assets/" + header + author + "-" + trailer + ".csv"
Y_predictions.to_csv(out_file, index=False)



In [20]:
# Si je souhaite insérer une fonction (ou une classe ) dans un pipeline de traitement, comment dois je l'écrire ?

# from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import FunctionTransformer

# # Définir la fonction à insérer dans le pipeline
# def custom_function(X):
#     # Faire quelque chose avec X
#     # Par exemple, ajouter 1 à chaque valeur
#     return X + 1

# # Créer un pipeline avec la fonction encapsulée dans FunctionTransformer
# pipeline = Pipeline([
#     ('custom_transformer', FunctionTransformer(custom_function))
# ])

# # Utiliser le pipeline sur les données
# X_transformed = pipeline.fit_transform(X)

In [21]:
# from sklearn.base import BaseEstimator, TransformerMixin

# class CustomTransformer(BaseEstimator, TransformerMixin):
#     def __init__(self, parameter1, parameter2):
#         self.parameter1 = parameter1
#         self.parameter2 = parameter2

#     def fit(self, X, y=None):
#         # Réaliser des opérations d'initialisation ou d'apprentissage ici
#         return self

#     def transform(self, X):
#         # Réaliser la transformation des données ici
#         transformed_X = X + self.parameter1
#         transformed_X *= self.parameter2
#         return transformed_X

# # Créer un pipeline avec la classe personnalisée
# pipeline = Pipeline([
#     ('custom_transformer', CustomTransformer(parameter1=1, parameter2=2))
# ])

# # Utiliser le pipeline sur les données
# X_transformed = pipeline.fit_transform(X)

In [22]:
# VOIR AUSSI

# # Création du pipeline avec des étapes de prétraitement et un estimateur final
# pipeline = Pipeline([
#     ('preprocessing', CustomTransformer()),  # Transformer personnalisé
#     ('estimator', RandomForestClassifier())  # Estimateur final
# ])

# # Apprentissage sur l'ensemble d'entraînement
# pipeline.fit(X_train, y_train)

# # Prédiction sur l'ensemble de test
# y_pred = pipeline.predict(X_test)